In [47]:
"""
ДОМАШНЕЕ ЗАДАНИЕ

Задача:

- разработать классификатор, который умеет отделять естественные тексты от сгенерированных, 
  по точности он должен работать "устойчиво лучше" чем мой "базовый" -- для всех я буду у себя запускать одну
  и ту же модель; оригинальный текст использовать, разумеется, нельзя;
  
- продемонстрировать, что вы умеете подбирать параметры с помощью GridSearch и оценивать качество (accuracy), 
  используя KFold;

- всё должно быть оформлено в виде отчёта с кодом в айпайтон-ноутбуке; результаты должны воспроизводиться и 
  выглядеть убедительно.

----

* Код при необходимости лучше разносить на разные клетки.

* Предварительный "исследовательский" анализ данных приветствуется, равно как и 
  придуманные самостоятельно фичи.

* Все отличия от исходного ноутбука нужно прокомментировать -- что и зачем вы делаете.

* В конце ноутбука должны быть итоги и ваши размышления о том, 
  ПОЧЕМУ при каких-то параметрах ваша модель не работала, а при "победных" сработала.

"""
%matplotlib inline

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.dummy import DummyClassifier
from sklearn import preprocessing
from sklearn.model_selection import KFold
import scipy as sp
import pandas as pd
import numpy as np

# может зависеть от версии sklearn
# from sklearn.grid_search import GridSearchCV

from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

df = pd.read_csv("works/anya.txt.csv")

print("Data shape:", df.shape)
print(df.groupby(["is_natural"]).size())

""" Длина предложения -- хороший ли признак в нашем случае?

df["sent_length"] = df["txt"].map(lambda x : len(x.split(" ")))
df["sent_length"].hist().plot()

print(df.groupby(["sent_length", "is_natural"]).size())
"""

# hint: разберитесь, что делает TfIdfVectorizer
# hint: посмотрите, какие ещё бывают векторизаторы для текстов
# hint: обязательно посмотрите, какие ещё у них бывают параметры
vectorizer = TfidfVectorizer(min_df=2)

# hint: можно надеяться не только на векторизатор, а сделать собственные фичи
# например, длина предложения -- чем не фича?

# hint: прочитайте в документации, что делает DummyClassifier
# hint: вам стоит выбрать более осмысленную модель : )
model = DummyClassifier(random_state=42)

# цепочка действий, которые будут произведены над данными
pipeline = Pipeline([
        # Мы могли бы и не склеивать пайплайн, а построить фичи для всех данных и только потом запустить
        # подбор гиперпараметров только для предсказывающей модели.
        # ----------------------------------------------------------------------------------------------
        # Но это было бы нечестно (то есть оценки были бы слишком оптимистичными),
        # так как значение tf-idf зависит от всего датасета, и это значит, 
        # надо подгонять "векторизатор" только по обучающей выборке, потому что 
        # "в реальной жизни" "тестовых данных" нет.
    ('vectorization',  vectorizer),
    ('classification', model),
])

# число разбиений
# ЭТО МЕНЯТЬ НЕ НУЖНО
n_fold = 3

# решётка параметров для всего пайплайна pipeline
# значения параметров, которые будем перебирать, когда будем искать лучшую модель
# classification__ -- это префикс, соответствующий model (см. pipeline)
# соответственно, для параметров векторизатора надо будет использовать другой (см. pipeline выше)
param_grid = [
  {'classification__strategy': ["most_frequent", "stratified", "prior", "uniform"]},
  {'classification__strategy': ["constant"], "classification__constant": [0, 1]},
 ]

# ознакомьтесь с параметрами KFold
cv = KFold(n_splits=n_fold, shuffle=True, random_state=42)
    
X_text = df["txt"]
y = df["is_natural"]

# посмотрите, что означает каждый параметр, и подберите подходящий
clf = GridSearchCV(estimator=pipeline, cv=cv, param_grid=param_grid, n_jobs=4, verbose=1, scoring="accuracy")
clf.fit(X_text, y) 

print("\nBest parameters set found on development set:\n")
print(clf.best_params_)
print("\n\nGrid scores on development set:\n")
means = clf.grid_scores_
print(means)

# best_model = clf.best_estimator_
# print("\nBest model:\t", best_model)

scorer = clf.scorer_
print("\nUsed scorer:\t", scorer)

# выберите модель, векторизатор, их параметры так, 
# чтобы эта оценка была выше высланной преподавателем
print("\nBest score:\t", clf.best_score_)

# Запустите клетку несколько раз. Значительно ли меняется результат?
# Подставьте другие random_state и проделайте то же самое. Значительно ли меняется результат?

Data shape: (3876, 2)
is_natural
0.0    1938
1.0    1938
dtype: int64
Fitting 3 folds for each of 6 candidates, totalling 18 fits


[Parallel(n_jobs=4)]: Done  18 out of  18 | elapsed:    2.0s finished



Best parameters set found on development set:

{'classification__strategy': 'stratified'}


Grid scores on development set:

[mean: 0.47730, std: 0.00813, params: {'classification__strategy': 'most_frequent'}, mean: 0.50722, std: 0.01376, params: {'classification__strategy': 'stratified'}, mean: 0.47730, std: 0.00813, params: {'classification__strategy': 'prior'}, mean: 0.48555, std: 0.03326, params: {'classification__strategy': 'uniform'}, mean: 0.50000, std: 0.02411, params: {'classification__strategy': 'constant', 'classification__constant': 0}, mean: 0.50000, std: 0.02411, params: {'classification__strategy': 'constant', 'classification__constant': 1}]

Used scorer:	 make_scorer(accuracy_score)

Best score:	 0.507223942208


/usr/local/lib/python3.4/dist-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)
